In [2]:
print "Importing packages"

import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as learn
import xgboost
import pandas as pd
import numpy as np
import re


Importing packages


Third attempt at the DonorsChoose Dataset. Random Forest. 

Anisotropic's notebook on Stacking\ensembling was really useful for creating this.

In [3]:
#reading data

train= pd.read_csv("/home/souravg/train.csv")

test=pd.read_csv("/home/souravg/test.csv") 

resource= pd.read_csv("/home/souravg/resources.csv")

print train.head(7)

print resource.head(7)

train.columns

print train.columns

train.school_state.astype("category")

train.project_grade_category.astype("category")

train.project_subject_categories.astype("category")

train.project_subject_subcategories.astype("category")

print train.groupby("school_state").size()

print train.groupby("project_grade_category").size()

print train.groupby("project_subject_categories").size()

print train.groupby("project_subject_subcategories").size()




/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        id                        teacher_id teacher_prefix school_state  \
0  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
1  p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
2  p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   
3  p185307  525fdbb6ec7f538a48beebaa0a51b24f            Mr.           NC   
4  p013780  a63b5547a7239eae4c1872670848e61a            Mr.           CA   
5  p063374  403c6783e9286e51ab318fba40f8d729           Mrs.           DE   
6  p103285  4e156c5fb3eea2531601c8736f3751a7           Mrs.           MO   

  project_submitted_datetime project_grade_category  \
0        2016-11-18 14:45:59          Grades PreK-2   
1        2017-04-26 15:57:28             Grades 3-5   
2        2017-01-01 22:57:44             Grades 3-5   
3        2016-08-12 15:42:11             Grades 3-5   
4        2016-08-06 09:09:11             Grades 6-8   
5        2016-11-05 10:01:51          Grades PreK-2   
6     

The cell above read the train data, converted appropriate parameters to categorical type & did some group by counts. From the results it seems like California, New Yrok, Texas & Florida have the largest number of projects.
Grades PreK-2 had the most number of projects, & the number of projects decrease monotonically with increasing age of students. Grades 9-12 have the fewest number of projects. I must say that this came as a surprise, I would hae thought seniors would need more funding.
There seems to be the following project categories:-

Applied Learning
Health & Sports
History & Civics
Math & Science
Music & The Arts
Special Needs
Warmth, Care & Hunger

But the majority of projects are tagged with multiple categories, hence there are lot more different project categories in the data (combinations of the base categories).

Math & Science, Literacy & Language & Health & Sport seem to be leaders in terms of project count.

In [4]:

ftest=test.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime'],1)

ftest.head(4)

#print target.head(5)

ftest["school_state"]=ftest["school_state"].astype('category')
ftest["project_grade_category"]=ftest["project_grade_category"].astype('category')
ftest["project_subject_categories"]=ftest["project_subject_categories"].astype('category')
ftest["project_subject_subcategories"]=ftest["project_subject_subcategories"].astype('category')

print ftest["school_state"].describe()
print ftest["project_grade_category"].describe()
print ftest["project_subject_categories"].describe()
print ftest["project_subject_subcategories"].describe()

cat_columns = ftest.select_dtypes(['category']).columns

print cat_columns

ftest[cat_columns] = ftest[cat_columns].apply(lambda x: x.cat.codes)

print ftest.head(5)


count     78035
unique       51
top          CA
freq      10723
Name: school_state, dtype: object
count             78035
unique                4
top       Grades PreK-2
freq              31471
Name: project_grade_category, dtype: object
count                   78035
unique                     51
top       Literacy & Language
freq                    16777
Name: project_subject_categories, dtype: object
count        78035
unique         390
top       Literacy
freq          6859
Name: project_subject_subcategories, dtype: object
Index([u'school_state', u'project_grade_category',
       u'project_subject_categories', u'project_subject_subcategories'],
      dtype='object')
   school_state  project_grade_category  project_subject_categories  \
0             4                       3                          40   
1            40                       0                          28   
2            40                       3                          32   
3            12                      

For the first attempt, decided to make a simple decision tree classifier

In [10]:
state_sum=train.groupby("school_state").size()

#print state_sum

ftrain=train.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime'],1)

ftrain.head(4)

target=ftrain.project_is_approved

ftrain=ftrain.drop(['project_is_approved'],1)

print target.head(5)

ftrain["school_state"]=ftrain["school_state"].astype('category')
ftrain["project_grade_category"]=ftrain["project_grade_category"].astype('category')
ftrain["project_subject_categories"]=ftrain["project_subject_categories"].astype('category')
ftrain["project_subject_subcategories"]=ftrain["project_subject_subcategories"].astype('category')

print ftrain["school_state"].describe()
print ftrain["project_grade_category"].describe()
print ftrain["project_subject_categories"].describe()
print ftrain["project_subject_subcategories"].describe()

print ftrain["project_subject_subcategories"].head(10)

cat_columns = ftrain.select_dtypes(['category']).columns

print cat_columns

ftrain[cat_columns] = ftrain[cat_columns].apply(lambda x: x.cat.codes)

print ftrain.head(5)



from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn import ensemble

clf= ensemble.RandomForestClassifier(n_estimators=500,criterion='entropy',max_depth=3,warm_start=True,)
#clf = AdaBoostClassifier(n_estimators=500, learning_rate= 0.4)

print "Hola"

clf.fit(ftrain, target)

print clf.feature_importances_

result=clf.predict_proba(ftest)
#scores = cross_val_score(clf, ftrain, target)

#print scores


print result.shape

print 'Finito'


0    1
1    0
2    1
3    0
4    1
Name: project_is_approved, dtype: int64
count     182080
unique        51
top           CA
freq       25695
Name: school_state, dtype: object
count            182080
unique                4
top       Grades PreK-2
freq              73890
Name: project_grade_category, dtype: object
count                  182080
unique                     51
top       Literacy & Language
freq                    39257
Name: project_subject_categories, dtype: object
count       182080
unique         407
top       Literacy
freq         15775
Name: project_subject_subcategories, dtype: object
0                                     Literacy
1                 Performing Arts, Team Sports
2       Applied Sciences, Literature & Writing
3                            Health & Wellness
4                            Health & Wellness
5    Character Education, Literature & Writing
6                            Health & Wellness
7      Early Development, Literature & Writing
8           

In [11]:
#Writing results to file 
    
r_df=pd.DataFrame(dict(id=test.id,project_is_approved=result[:,1]))
        
print r_df.head(10)
r_df.to_csv('/home/souravg/output2.1.csv',index=False)
count=0

for i in result[:,1]:
    if i<.5:
        count+=1
print count
    


        id  project_is_approved
0  p233245             0.822630
1  p096795             0.853396
2  p236235             0.813079
3  p233680             0.849476
4  p171879             0.823579
5  p016071             0.853235
6  p099906             0.880715
7  p200236             0.844349
8  p129452             0.881523
9  p186652             0.851150
0
